In [2]:
!pip install --quiet gradio



In [4]:
import gradio as gr
import json, random, os
from pathlib import Path

BASE = Path("/content")
USERS_FILE = BASE / "users.json"
QUOTES_FILE = BASE / "quotes.json"


quotes = [
    {"id":1, "text":"The best way to get started is to quit talking and begin doing.", "author":"Walt Disney", "category":"inspire"},
    {"id":2, "text":"Don’t let yesterday take up too much of today.", "author":"Will Rogers", "category":"inspire"},
    {"id":3, "text":"Time is money. Waste it wisely.", "author":"Anonymous", "category":"money"},
    {"id":4, "text":"Love grows more tremendously full, swift, poignant, as the years multiply.", "author":"Zane Grey", "category":"anniversary"},
    {"id":5, "text":"Anger is one letter short of danger.", "author":"E. S. White", "category":"angry"},
    {"id":6, "text":"Tears come from the heart and not from the brain.", "author":"Leonardo da Vinci", "category":"sad"},
    {"id":7, "text":"Count your life by smiles, not tears.", "author":"John Lennon", "category":"happy"},
    {"id":8, "text":"Birthdays are nature’s way of telling us to eat more cake.", "author":"Anonymous", "category":"birthday"},
    {"id":9, "text":"Awkward moments build character.", "author":"Anonymous", "category":"awkward moment"},
    {"id":10, "text":"Success usually comes to those who are too busy to be looking for it.", "author":"Henry David Thoreau", "category":"success"},
    {"id":11, "text":"It is better to be alone than in bad company.", "author":"George Washington", "category":"alone"}
]
users = {"users": []}

with open(QUOTES_FILE, "w") as f: json.dump(quotes, f, indent=2)
with open(USERS_FILE, "w") as f: json.dump(users, f, indent=2)


def load(path): return json.load(open(path))
def save(path, data): json.dump(data, open(path, "w"), indent=2)

def signup(username, password):
    db = load(USERS_FILE)
    if any(u["username"] == username for u in db["users"]):
        return "User already exists", None
    db["users"].append({"username": username, "password": password})
    save(USERS_FILE, db)
    return "Signup successful! Logged in.", username

def login(username, password):
    db = load(USERS_FILE)
    if any(u["username"] == username and u["password"] == password for u in db["users"]):
        return "Login successful!", username
    return "Invalid username/password", None

def show_quotes(category):
    db = load(QUOTES_FILE)
    if category == "all": return "\n".join([q["text"] for q in db])
    return "\n".join([q["text"] for q in db if q["category"] == category])

def random_quote(category):
    db = load(QUOTES_FILE)
    filtered = db if category == "all" else [q for q in db if q["category"] == category]
    return random.choice(filtered)["text"]

def add_quote(text, author, category, user):
    if not user: return "Login required!"
    db = load(QUOTES_FILE)
    new_id = max([q["id"] for q in db]) + 1
    db.append({"id": new_id, "text": text, "author": author, "category": category})
    save(QUOTES_FILE, db)
    return "Quote added!"


with gr.Blocks() as app:
    user = gr.State()

    gr.Markdown("## Quote of the Day App")

    with gr.Tab("Login / Signup"):
        u = gr.Textbox(label="Username")
        p = gr.Textbox(label="Password", type="password")
        login_btn = gr.Button("Login")
        signup_btn = gr.Button("Signup")
        out = gr.Textbox(label="Status", interactive=False)

        login_btn.click(login, [u, p], [out, user])
        signup_btn.click(signup, [u, p], [out, user])

    with gr.Tab("Browse Quotes"):
        category = gr.Dropdown(
            ["all","inspire","money","anniversary","angry","sad","birthday","awkward moment","success","alone","happy"],
            label="Choose category"
        )
        show_btn = gr.Button("Show Quotes")
        random_btn = gr.Button("Random Quote")
        q_out = gr.Textbox(label="Quote", lines=6)

        show_btn.click(show_quotes, [category], q_out)
        random_btn.click(random_quote, [category], q_out)

    with gr.Tab("Add Quote"):
        q_text = gr.Textbox(label="Quote text")
        q_author = gr.Textbox(label="Author")
        q_category = gr.Textbox(label="Category")
        add_btn = gr.Button("Add")
        add_out = gr.Textbox(label="Status")

        add_btn.click(add_quote, [q_text, q_author, q_category, user], add_out)

app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://936a07542b3b1600b7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
